In [ ]:
#importing libraries
from google.colab import drive
from keras.models import Model
from keras import models
from keras.layers import Input , LSTM , Dense
import tensorflow as tf
import pandas as pd
import numpy as np
drive.mount('/content/drive')

data_path = '/content/drive/My Drive/NLP-Sato/fra.txt'#data txt file path
batch_size = 64 #Batch size for training
epochs = 100 # Number of epochs to train for.
num_samples = 10000 #Number of samples to train on.
latent_dim = 256 #euuh

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#vectorizing the data
input_texts = [] #french input
target_texts = [] #english target output
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding = 'utf-8') as f:
  lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines)-1)]:
  target_text,input_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)

In [ ]:
input_text

In [ ]:
target_characters = sorted(list(target_characters))
input_characters = sorted(list(input_characters))
number_encoder_tokens = len(input_characters)
number_decoder_tokens = len(target_characters)
max_encoder_seq_length = max ([len(txt) for txt in input_texts])
max_decoder_seq_length = max ([len(txt) for txt in target_texts])

In [ ]:
max_decoder_seq_length

17

In [ ]:
print('Number of samples:',len(input_texts))
print('Number of unique input tokens',number_encoder_tokens)
print('Number of unique output tokens:',number_decoder_tokens)
print('Max sequence length for inputs:',max_encoder_seq_length)
print('Max sequence length for outputs:',max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens 90
Number of unique output tokens: 73
Max sequence length for inputs: 57
Max sequence length for outputs: 17


In [ ]:
target_characters

['\t',
 '\n',
 ' ',
 '!',
 '"',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é']

In [ ]:
input_token_index = dict(
    [(char,i) for i, char in enumerate(input_characters)]
)
target_token_index = dict(
    [(char,i) for i, char in enumerate(target_characters)]
)

In [ ]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '"': 4,
 '$': 5,
 '%': 6,
 '&': 7,
 "'": 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '5': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'W': 44,
 'Y': 45,
 'a': 46,
 'b': 47,
 'c': 48,
 'd': 49,
 'e': 50,
 'f': 51,
 'g': 52,
 'h': 53,
 'i': 54,
 'j': 55,
 'k': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'w': 68,
 'x': 69,
 'y': 70,
 'z': 71,
 'é': 72}

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, number_encoder_tokens), dtype='float32'
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, number_decoder_tokens),dtype='float32'
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, number_decoder_tokens),dtype='float32'
)

In [ ]:
target_text

'\tI can hear you.\n'

In [ ]:
#transform encode/decode target and input in matricies zeroes and ones
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]] = 1
  encoder_input_data[i,t + 1:,input_token_index[' ']] = 1
  for t, char in enumerate(target_text):
    decoder_input_data[i,t, target_token_index[char]] = 1
    if t > 0:
      decoder_target_data[i, t - 1, target_token_index[char]] = 1
  decoder_input_data[i, t + 1:, target_token_index[' ']] = 1
  decoder_target_data[i, t:,target_token_index[' ']] = 1

In [ ]:
encoder_input_data1 = np.zeros(
    (1, max_encoder_seq_length, number_encoder_tokens), dtype='float32'
)
def transform(input_text):
  for t,char in enumerate(input_text):
    encoder_input_data1[:,t,input_token_index[char]] =1
  encoder_input_data1[:,t+1:,input_token_index[' ']] = 1
  return encoder_input_data1

In [ ]:
print(encoder_input_data1.shape)

(1, 57, 90)


In [ ]:
encoder_inputs = Input(shape=(None, number_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None,number_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,initial_state = encoder_states)
decoder_dense = Dense(number_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
#run training
#model.summary()
#defining model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999, decay=0.001), loss='categorical_crossentropy',
              metrics=["accuracy"])


In [ ]:
model.fit(
    [encoder_input_data, decoder_input_data], 
    decoder_target_data,
    batch_size = batch_size,
    epochs = epochs,
    validation_split=0.2)
model.save("E2S")

Epoch 1/100
125/125 [==============================] - 53s 402ms/step - loss: 2.0345 - accuracy: 0.4572 - val_loss: 1.9513 - val_accuracy: 0.4303
Epoch 2/100
125/125 [==============================] - 49s 396ms/step - loss: 1.3756 - accuracy: 0.5957 - val_loss: 1.6223 - val_accuracy: 0.5038
Epoch 3/100
125/125 [==============================] - 49s 392ms/step - loss: 1.2014 - accuracy: 0.6386 - val_loss: 1.4981 - val_accuracy: 0.5484
Epoch 4/100
125/125 [==============================] - 50s 398ms/step - loss: 1.0566 - accuracy: 0.6732 - val_loss: 1.5560 - val_accuracy: 0.5450
Epoch 5/100
125/125 [==============================] - 50s 399ms/step - loss: 0.9609 - accuracy: 0.7002 - val_loss: 1.4424 - val_accuracy: 0.5746
Epoch 6/100
125/125 [==============================] - 51s 411ms/step - loss: 0.8783 - accuracy: 0.7243 - val_loss: 1.4072 - val_accuracy: 0.5941
Epoch 7/100
125/125 [==============================] - 50s 404ms/step - loss: 0.8084 - accuracy: 0.7456 - val_loss: 1.3719 -

INFO:tensorflow:Assets written to: E2S/assets


INFO:tensorflow:Assets written to: E2S/assets


In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='modelsummary.png', show_shapes=True, show_layer_names=True)

In [ ]:
model.save("/content/drive/My Drive/NLP-Sato/MyModel")

INFO:tensorflow:Assets written to: /content/drive/My Drive/NLP-Sato/MyModel/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/NLP-Sato/MyModel/assets


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items()
)

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, number_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    #target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, number_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence
#encoder_input_data[seq_index: seq_index + 1]
for seq_index in range(100):
  input_seq = encoder_input_data[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)
  print('--')
  print('Input sentence : ',input_texts[seq_index])
  print('Decoded sentence : ', decoded_sentence)
#input_seq.shape
  #return model

In [ ]:
#model=models.load_model("/content/drive/My Drive/NLP-Sato/MyModel")
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1, number_decoder_tokens))
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        target_seq = np.zeros((1, 1, number_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]
    return decoded_sentence

#i = np.random.choice(len(input_texts))
#input_seq = encoder_input_data[i:i+1]
input_text_test="Vas-y."
input_seq= transform(input_text_test)
translation = decode_sequence(input_seq)
print('-')
print('Input:', input_text_test)
print('Translation:', translation)

NameError: ignored